Este notebook muestra los pasos para cargar el modelo **[DeepESP/gpt2-spanish](https://huggingface.co/DeepESP/gpt2-spanish)**  desde HuggingFace, cargar el dataset, entrenar el modelo y finalmente almacenar el modelo modificado en Google Drive

**Objetivo:** Cargar modelo base, preparar datos, hacer fine-tuning y probar inferencias manuales.

Contenido:

🔹 Montar Google Drive.

🔹 Cargar modelo base desde Hugging Face (tokenizer + modelo).

🔹 Cargar dataset de entrenamiento desde Drive.

🔹 Definir y ejecutar función de fine-tuning.

🔹 Guardar modelo ajustado en Google Drive.


In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, DataCollatorForLanguageModeling
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset
from google.colab import drive
import tqdm
import torch
import json
import os

In [2]:
print(f"¿CUDA disponible?: {torch.cuda.is_available()}")
print(f"Nombre de la GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'No GPU'}")
print(f"Número de GPUs: {torch.cuda.device_count()}")

¿CUDA disponible?: True
Nombre de la GPU: Tesla T4
Número de GPUs: 1


## Cargar modelo

🔹 Se establecen las rutas de Google Drive donde se guardará el modelo y donde se lee el dataset:


In [ ]:
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')
else:
    print("Drive ya está montado")

# Ruta base en Google Drive donde se guarda el modelo
BASE_PATH = "/content/drive/MyDrive/ProyectoFinal"

# Crear subcarpeta específica para este modelo y método
VERSION = "full_fine_tuning_v4"
MODEL_SAVE_PATH = f"{BASE_PATH}/FineTuning/DeepESP_gpt2-spanish/{VERSION}"
DATA_FOLDER = f"{MODEL_SAVE_PATH}/Data" # Carpeta donde están los archivos .json

# Crear la carpeta si no existe
os.makedirs(MODEL_SAVE_PATH, exist_ok=True)

Mounted at /content/drive


---
🔹 Se carga el modelo desde HuggingFace

In [3]:
# Cargar el tokenizer y el modelo
# Definir dispositivo
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando dispositivo: {device}")

# Verificar disponibilidad de CUDA
if not torch.cuda.is_available() and device == "cuda":
    raise RuntimeError("CUDA no está disponible. Asegúrate de que la GPU esté habilitada en Colab.")

# Cargar el Modelo original
print("Cargando modelo original...")
MODEL_NAME = "DeepESP/gpt2-spanish"
tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)

# Agregar tokens especiales
tokenizer.add_special_tokens({
    "pad_token": "<pad>",
    "bos_token": "<bos>",
    "eos_token": "<eos>"
})
tokenizer.add_tokens(["<art>", "<lyr>"])

model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)
model.resize_token_embeddings(len(tokenizer))


# Mover el modelo al dispositivo
model = model.to(device)
print(f"Modelo en: {next(model.parameters()).device}")

# Verificar que los tokens especiales estén correctamente configurados:
print(f"Tokens especiales: {tokenizer.special_tokens_map}")
print(f"ID de <lyr>: {tokenizer.encode('<lyr>')}")
print(f"ID de <art>: {tokenizer.encode('<art>')}")
print(f"ID de <eos>: {tokenizer.encode('<eos>')}")

# Después de cargar model y tokenizer
print(f"Tamaño vocabulario tokenizer: {len(tokenizer)}")
print(f"Tamaño embeddings modelo: {model.get_input_embeddings().weight.shape[0]}")


Usando dispositivo: cuda
Cargando modelo original...


tokenizer_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/262 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/914 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/261M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Modelo en: cuda:0
Tokens especiales: {'bos_token': '<bos>', 'eos_token': '<eos>', 'unk_token': '<|endoftext|>', 'pad_token': '<pad>', 'additional_special_tokens': ['<|talk|>', '<|ax1|>', '<|ax2|>', '<|ax3|>', '<|ax4|>', '<|ax5|>', '<|ax6|>', '<|ax7|>', '<|ax8|>', '<|ax9|>']}
ID de <lyr>: [50261]
ID de <art>: [50260]
ID de <eos>: [50259]
Tamaño vocabulario tokenizer: 50262
Tamaño embeddings modelo: 50262


In [ ]:
# Función auxiliar para determinar si una letra

def es_muy_corta(letra, min_lineas=3, min_caracteres=100):
    """
    Devuelve True si la letra es considerada 'muy corta',
    es decir, tiene pocas líneas y pocos caracteres.
    """
    lineas = [l for l in letra.strip().split("\n") if l.strip() != ""]
    return len(lineas) < min_lineas or len(letra.strip()) < min_caracteres

In [ ]:
# Dataset personalizado para las letras de canciones

class LyricsDataset(Dataset):
    def __init__(self, folder_path, tokenizer, max_length=1024):
        self.samples = []

        # Leer todos los archivos .json de la carpeta
        for filename in os.listdir(folder_path):
            if filename.endswith(".json"):
                filepath = os.path.join(folder_path, filename)
                print(f"Leyendo archivo: {filepath}")  # Mostrar el archivo que se está leyendo
                with open(filepath, "r", encoding="utf-8") as f:
                    artist_data = json.load(f)
                    artist_name = artist_data.get("name", "Unknown")
                    for song in artist_data.get("songs", []):
                        lyric = song.get("lyric", "").strip()
                        if lyric:
                          if es_muy_corta(letra=lyric, min_lineas=4, min_caracteres=150):
                              formatted = f"<bos><art>{artist_name}<lyr>{lyric}"
                          else:
                              formatted = f"<bos><art>{artist_name}<lyr>{lyric}<eos>"
                          self.samples.append(formatted)

        # Tokenizar todos los samples
        self.encodings = tokenizer(self.samples, max_length=max_length, truncation=True, padding="max_length", return_tensors="pt")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        # return (self.encodings['input_ids'][idx], self.encodings['attention_mask'][idx])
        item = {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx]
        }
        return item

In [ ]:
# Función de entrenamiento (Full fine tuning)

def fullFineTuning():
    # Crear dataset y dataloader
    dataset = LyricsDataset(DATA_FOLDER, tokenizer)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False  # Importante para modelos causales como GPT-2
    )

    dataloader = DataLoader(
        dataset,
        batch_size=8,
        shuffle=True,
        collate_fn=data_collator,
    )

    # Preparar optimizador (antes 5e-2)
    optim = Adam(model.parameters(), lr=2e-5)

    # Entrenamiento
    epochs = 5 # antes 3
    epoch_losses = []

    print("Comenzando entrenamiento completo...")
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        loop = tqdm.tqdm(dataloader, leave=True)
        for batch in loop:

            # El batch del DataCollator es un diccionario
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device) # Estos labels ya están listos

            optim.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optim.step()

            total_loss += loss.item()
            loop.set_description(f"Epoch {epoch+1}")
            loop.set_postfix(loss=loss.item())

        average_loss = total_loss / len(dataloader)
        epoch_losses.append(average_loss)
        print(f"Epoch {epoch+1} finalizado con loss promedio: {average_loss:.4f}")

        # weights_path = os.path.join(MODEL_SAVE_PATH, f"model_epoch_{epoch+1}.pt")
        # torch.save(model.state_dict(), weights_path)
        # print(f"Pesos guardados en {weights_path}")

    model.save_pretrained(MODEL_SAVE_PATH)
    tokenizer.save_pretrained(MODEL_SAVE_PATH)
    print(f"Modelo completo guardado en {MODEL_SAVE_PATH}")

    # Guardar un log de entrenamiento
    training_log = {
        "modelo": MODEL_NAME,
        "metodo": "Full fine tuning",
        "epochs": epochs,
        "losses": epoch_losses  # Esto lo explico abajo
    }

    log_path = os.path.join(MODEL_SAVE_PATH, "training_log.json")
    with open(log_path, "w", encoding="utf-8") as f:
        json.dump(training_log, f, ensure_ascii=False, indent=4)

    print(f"Training log guardado en {log_path}")


In [ ]:
# ---- Ejecución principal ----

fullFineTuning()


Leyendo archivo: /content/drive/MyDrive/ProyectoFinal/FineTuning/DeepESP_gpt2-spanish/full_fine_tuning_v4/Data/05-palito-ortega-484.json
Leyendo archivo: /content/drive/MyDrive/ProyectoFinal/FineTuning/DeepESP_gpt2-spanish/full_fine_tuning_v4/Data/19-fito-paez-357.json
Leyendo archivo: /content/drive/MyDrive/ProyectoFinal/FineTuning/DeepESP_gpt2-spanish/full_fine_tuning_v4/Data/17-ulises-bueno-385.json
Leyendo archivo: /content/drive/MyDrive/ProyectoFinal/FineTuning/DeepESP_gpt2-spanish/full_fine_tuning_v4/Data/18-jairo-353.json
Leyendo archivo: /content/drive/MyDrive/ProyectoFinal/FineTuning/DeepESP_gpt2-spanish/full_fine_tuning_v4/Data/03-juan-gabriel-492.json
Leyendo archivo: /content/drive/MyDrive/ProyectoFinal/FineTuning/DeepESP_gpt2-spanish/full_fine_tuning_v4/Data/04-raphael-489.json
Leyendo archivo: /content/drive/MyDrive/ProyectoFinal/FineTuning/DeepESP_gpt2-spanish/full_fine_tuning_v4/Data/02-la-barra-528.json
Leyendo archivo: /content/drive/MyDrive/ProyectoFinal/FineTuning/D

Epoch 1: 100%|██████████| 1084/1084 [19:55<00:00,  1.10s/it, loss=2.02]


Epoch 1 finalizado con loss promedio: 2.8578


Epoch 2: 100%|██████████| 1084/1084 [19:54<00:00,  1.10s/it, loss=2.96]


Epoch 2 finalizado con loss promedio: 2.5639


...

Epoch 3 finalizado con loss promedio: 2.4353


Epoch 4: 100%|██████████| 1084/1084 [19:54<00:00,  1.10s/it, loss=2.36]


Epoch 4 finalizado con loss promedio: 2.3336


Epoch 5: 100%|██████████| 1084/1084 [19:54<00:00,  1.10s/it, loss=2.39]


Epoch 5 finalizado con loss promedio: 2.2377
Modelo completo guardado en /content/drive/MyDrive/ProyectoFinal/FineTuning/DeepESP_gpt2-spanish/full_fine_tuning_v4
Training log guardado en /content/drive/MyDrive/ProyectoFinal/FineTuning/DeepESP_gpt2-spanish/full_fine_tuning_v4/training_log.json
